In [10]:
from langchain_classic.storage import InMemoryStore
from langchain_classic.retrievers.parent_document_retriever import ParentDocumentRetriever
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

In [2]:
loaders = [TextLoader('./data/How_to_invest_money.txt', encoding='utf-8')]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [8]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)
embeddings = HuggingFaceEmbeddings(model='BAAI/bge-m3', model_kwargs={'device':'cuda'})
vectorstore = Chroma(collection_name='split_parents', embedding_function=embeddings)
store = InMemoryStore()

In [9]:
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter
)
retriever.add_documents(docs)
print(len(list(store.yield_keys())))

219


In [11]:
def create_virtual_doc_chain():
    system = '당신은 고도로 숙련된 AI입니다.'
    user = """주어진 질문 '{query}'에 대해 직접적으로 답변하는 가상의 문서를 생성하세요. 문서의 크기는 {chunk_size} 글자 언저리여야 합니다."""
    prompt = ChatPromptTemplate.from_messages(
        [('system', system), ('human', user)]
    )
    model = ChatOpenAI(model='gpt-5-nano', temperature=0)
    return prompt | model | StrOutputParser()

In [13]:
def create_retrieval_chain():
    return RunnableLambda(lambda x: retriever.invoke(x['virtual_doc']))

def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

In [14]:
def create_final_response_chain():
    final_prompt = ChatPromptTemplate.from_template("""
다음 정보와 질문을 바탕으로 답변해주세요:
                                                    컨텍스트: {context}
                                                    질문: {question}
                                                    답변:""")
    final_llm = ChatOpenAI(model='gpt-5-nano', temperature=0)
    return final_prompt | final_llm | StrOutputParser()

In [15]:
def print_input_output(input_data, output_data, step_name):
    print(f'\n--- {step_name} ---')
    print(f'Input: {input_data}')
    print(f'Output: {output_data}')
    print('-' * 50)

In [ ]:
def create_pipeline_with_logging():
    virtual_doc_chain = create_virtual_doc_chain()
    retrieval_chain = create_retrieval_chain()
    final_response_chain = create_final_response_chain()

    # 아래에 이어가야